# Практический обзор основных задач анализа временных рядов

В данном обзоре мы разберем основные задачи анализа временных рядов, такие как:

* [Статистический анализ](#statistical_analysis)
* [Прогнозирование](#forecasting)
* [Поиск аномалий](#anomaly_detection)
* [Классификация временных рядов](#time_series_classification)

Обзор содержит краткую теорию, достаточную для решения задач, и, практические задачи,
которые по большей части требуют написания кода. Такие задачи вы сможете найти по
комментарию `# <your_code_here>`. 

Список рекомендуемой литературы вы сможете найти в конце обзора.

Удачи!


# <a id='statistical_analysis'>Статистический анализ</a>

Прежде чем строить какие-либо модели, бывает крайне полезно посмотреть на ваши временные
ряды, покрутить их "туда-сюда", чтобы понять их основные характеристики. Это поможет вам
лучше понять, какой подход использовать или свести задачу к гораздо более простой. Иногда,
вдумчивый статистический подход сам по себе может являться моделью прогнозирования или
поиска аномалий, способную опередить ML и DL подходы.

Данная секция состоит из следующих частей:

1. [Знакомимся с основными понятиями](#basic_concepts)
2. [Преобразования временных рядов](#time_series_transformations)
3. [Как преобразовать ряд к стационарному](#making_series_stationary)
4. [Разложения временных рядов, STL разложение](#time_series_decomposition)

## <a id='basic_concepts'>Знакомимся с основными понятиями</a>


### *Что такое временной ряд?*  


Временным рядов является любая упорядоченная последовательность чисел (или набора чисел),
взятая с равными промежутками времени. На практике, мы всегда имеем дело с пропусками в
данных, поэтому какими-то пропусками в данных приходится пренебрегать. Тем не менее, если
вы имеете дело с числами, взятыми с совсем случайными интервалами, вы уже переходите в
область стохастических процессов и теория, приведенная в данном обзоре, будет плохо к ним
применима.

Итак, временной ряд формально:

$$
ts = \{(t_i, v_i): i \in {0...N}, v_i \in R^N, t_{i+1} - t_i = C \: \forall i\}
$$

В питоне наиболее подходящей структурой данных для работы с временным рядов является 
`pandas.Series` (для хранения многомерных временных рядов используйте `DataFrame`)

In [ ]:
import pandas as pd
from random import randint

ts = pd.Series(
    data=[randint(0, 100) for _ in range(100)],
    index=pd.date_range(start='2022-08-05', freq='1H', periods=100)
)

`data` любой итерируемый обьект
`index` также любой итерируемый обьект, равный по длине `data`. Для задач временных рядов 
нам необходимо, чтобы он состоял из обьектов datetime, создать которые можно либо
средствами стандартной библиотеки datetime либо средствами pandas. Обратите внимание, что,
чтобы задавать временные интервалы, нужно использовать стандарт ISO8601 (подробнее [здесь](#ref_1))

Изучите самостоятельно многообразие доступных методов класса pd.Series. 

В нашем туториале мы будем использовать разные временные ряды в качестве примеров. Для
удобной работы с ними, вы можете использовать класс `Dataset` из модуля `dataset` в корне
вашего репозитория.

In [ ]:
from dataset import Dataset

ds = Dataset('./data/dataset')

# посмотреть временные ряды
print(ds)

# достать конкретный временной ряд
ds['international-airline-passengers.csv']

# проитерироваться по всем рядам
for key, s in ds:
    pass

### *Графики временных рядов*

Любой анализ временных рядов предполагает визуализацию данных, для этого нам необходимо
уметь рисовать графики временных рядов. Самый простой способ - использовать встроенный
метод `plot` обьекта series.

In [ ]:
ts.plot()

Тем не менее, есть более интерактивные варианты для отрисовки графиков, такие как
библиотеки bokeh и plotly, например. Можете использовать функцию `plot_ts` из корня вашего
репозитория, как пример.

Обратите внимение, насколько информативнымии становятся графики!

In [ ]:
from plotting import plot_ts

plot_ts(ts)

### *Гранулярность временного ряда или частота*

Гранулярностью временного ряда (или частотой) называется временной интервал между двумя
соседними точками временного ряда. Она показывает, насколько "плотно" распределены точки в
истории.

In [ ]:
print((ts.index[1] - ts.index[0]))

### *Лаг(-и) временного ряда*

Работая с временными рядами, вы будете часто слышать термин `лаг`. Данный термин может
быть употребим либо по отношению ко всему временному ряду, либо по отношению к конкретной
точке $(t_i, v_i)$.

Когда термин лаг используется в контексте точки, `k-ый лаг` для
точки $(t_i, v_i)$ означает точку $(t_{i-k}, v_{i-k})$ вглубь истории.

Будучи применен ко всему ряду `k-ый лаг` означает ряд, сдвинутый `k` раз вправо (назад во
времени), таким образом, что каждая точка $(t_i, v_i)$ становится точкой $(t_i, v_{i-k})$

In [ ]:
plot_ts(ts, ts.shift(10))

### *Автокорреляция*

Практически во всех задачах анализа временных рядов важно знать, содержит ли ваш ряд 
какую-либо временную зависимость (т.е. коррелируют ли точки из прошлого с точками из будущего),
потому что прогнозирование временных рядов без доп признаков основано исключительно
на прошлых значениях временного ряда. Одним из лучших способов найти такую зависимость 
является расчет автокорреляции, т.е. корреляции между изначальным временным рядом и рядом,
сдвинутым назад во времени.

Говоря математически, автокорреляция ряда $Y_t$ при $k$-ом лаге определяется как хорошо
известная корреляция Пирсона, взятая от двух рядов, изначального и взятого с лагом $k$.

$$\rho_{Y_t, Y_{t-k}} = \frac{\operatorname{cov}({Y_{t-k}},Y_t)}{\sigma_{Y_{t-k}} \sigma_{Y_t}} = \frac{E[(Y_t-\mu_{Y_t})(Y_{t-k}-\mu_{Y_{t-k}})]}{\sigma_{Y_{t-k}} \sigma_{Y_t}}$$

Большие значения автокорреляции на каком либо лаге $k$ сигнализируют, что вы, возможно,
имеете сильную периодческую зависимость с периодом $1/k$.

In [ ]:
# в питоне, вы можете легко посчитать корреляцию при помощи метода `corr` обьекта series

ts.corr(ts.shift(10))

### *Частичная автокорреляция*

Когда мы расчитываем автокорреляцию ряда $Y_t$ при лаге $K$, мы подразумеваем, что все
промежуточные лаги тоже могут вносить определенный вклад в разброс значений $Y_t$. 
Но, что если мы хотим узнать, как зависит ряд в текущий момент времени от лага $K$,
если убрать влияние всех промежуточных лагов? Тогда, мы должны сначала построить линейную
регрессию ряда $Y_t$ на все промежуточные значения $Y_{t-1}$, $Y_{t-2}$, ..., $Y_{t-K+1}$.
После чего вычесть прогноз регрессии из исходного ряда. Тогда, автокорреляция полученного
ряда при лаге $K$ будет являться частичной автокорреляция исходного ряда при этом же лаге.

Формально, частичная корреляция определяется следующим образом:

$$
pacf(y_t, y_{t-k}) =
\begin{cases}
    corr(y_{t-k}, y_t) , k=1,\\
    corr(y_{t-k} - y_{t-k}^{'}, y_t - y_t^{'}),k>1,
\end{cases},
$$

где 
$y_{t}^{'}$ - линейная регрессия на $y_{t-1}, y_{t-2}, ..., y_{t-k+1}$,  
т.е. $y_{t}^{'} = \alpha_1 y_{t-1}^{'} + \alpha_2 y_{t-2}^{'} + ... + \alpha_{k-1} y_{t-k+1}^{'}$
и аналогично для $y_{t-k}^{'}$

In [ ]:
# в питоне частичную автокорреляция можно посчитать при помощи функции pacf из statsmodels
from statsmodels.tsa.stattools import pacf

pacf(ts, 30)

### *Графики acf и pacf*

Зависимость значения автокорреляции (частичной автокорреляции) от порядка лага называется
графиком автокорреляции или графиком частичной автокорреляции.

Графики автокорреляции используются для того, чтобы посмотреть, есть ли у временного ряда
какая либо временная зависимость. Например, для ряда белого шума, график автокорреляции
будет выглядеть как шум, без значимых пиков, а для ряда $sin(t)$ график автокорреляции
будет выглядеть как $sin(t)$

In [ ]:
# в питоне готовые функции для отрисовки можно найти в библиотеке statsmodels
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(ts, lags=40);
plot_pacf(ts, lags=40);

На графиках выше можно заметить, что какая либо структура в данных отсуствуют, т.к. мы
наблюдаем 100% корреляцию при нулевом лаге (когда ряд совпадает сам с собой) и случайные
корреляции на всех остальных лагах. Т.к. мы изначально генерировали случайный ряд, такой
результат очевиден.

### *Компоненты временного ряда*

Каждый временной ряд может содержать одну (или несколько) следующих компонент
временного ряда:

* гетероскедастичность
* тренд
* сезонность
* цикличность
* шум

#### Гетероскедастичность

Непостояноство дисперсии временного ряда называется гетероскедастичностью, на примере ниже
можно видеть, как ряд имеет возрастающую дисперсию.

In [ ]:
ts = ds['alcohol_sales.csv']

# первый график показывает изначальный ряд, второй график - скользящее std
plot_ts(ts, ts.rolling(12).std())

#### Тренд

У понятия тренда нет четкого формального определения, в целом его можно охарактеризовать
как долговременную тенденцию к изменению временного ряда. Часто, когда говорят о тренде,
имеют ввиду линейный тренд, но он вообще говоря может быть полиномом произвольной степени.

Обнаружить тренд легко можно на самом графике временного ряда либо на графике 
автокорреляции, ряды с трендом на нем выглядят как убывающая или возрастающая (зачастую
монотонно) зависимость.

In [ ]:
ts = ds['international-airline-passengers.csv']

# видим линейно автокорреляцию с ростом порядка лага, что свидетельствует о наличии тренда
plot_acf(ts, lags=70);

#### Сезонность и период сезонности временного ряда

Если $y_t = y_{t-s} + e(t) \forall t, \text{где e(t) - шум}$, то говорят, что временной
ряд имеет сезонность с периодом s. (*примечание: тут мы подразумеваем, что мы исключили
все остальные компоненты, включаю другие сезонности).
Определить сезонность можно либо по графику самого временного ряда, либо по графику
автокорреляции.
Если на графике автокорреляции вы наблюдаете периодические всплески автокорреляции через
равные промежутки времени, то ряд скорее всего имеет сезонность.

In [ ]:
ts = ds['daily-min-temperatures.csv']

plot_ts(ts)
plot_acf(ts);

#### Цикличность

Если временной ряд имеет повторяющийся паттерн, но, в отличии от сезонности, период его не
является постоянным, то мы имеем дело с цикличностью.

#### Шум

Если мы исключим из временного ряда все вышеперечисленные компоненты, то все, что у нас
останется, является случайной частью временного ряда - неким шумом из некоего распределения.

В эту же компоненту входят выбросы и различные аномалии.

График автокорреляций для шума не должен содержать каких-либо значимых автокорреляций.

In [ ]:
from random import randint

ts = pd.Series(
    data=[randint(0, 100) for _ in range(100)],
    index=pd.date_range(start='2022-08-05', freq='1H', periods=100)
)

plot_ts(ts)
plot_acf(ts);

### Понятие стационарности временного ряда

Одним из ключевых понятий в классическом, эконометрическом анализе временных рядов
является понятие стационарности. Познакомимся с ним подробнее.

#### Определение

Формально временной ряд $y_1, y_2, ..., y_t$ называется стационарным, если $\forall s$
распределение $y_t, ..., y_{t+s}$ не зависит от времени, а именно:

* $VAR(y_t) = const$
* $E(y_t) = const$
* $COV(y_t, y_{t-k}) = const \forall k$

#### Интуитивное определение

Говоря неформальным языком, стационарным временным рядом является такой временной ряд,
который мы избавили от любой формы "непостоянства". 

По сути, все компоненты временного ряда, за исключением шума, представляют собой ту или 
иную форму "непостоянства" во временном ряде. Гетероскедастичность показывает
непостоянство дисперсии, тренд и сезонность - изменчивость матожидания. Как мы узнаем
немного позже, умение "убирать" из временного ряда все возможные формы непостоянства -
ключевой навык для построения классических моделей прогнозирования. Сделать это можно при
помощи различных преобразований временного ряда.

#### Как убедиться, что ряд стационарен

Есть два основных способа убедиться в стационарности временного ряда.

1. Используя визуальный анализ исходного графика и графика автокорреляции.
2. Используя статистические тесты.

*Визуальный анализ*

При визуальном анализе мы обращаем внимание на то, есть ли у временного ряда непостоянные
компоненты - гетероскедастичность, тренд, сезонность. Обратите внимание, что цикличность
не является компонентной непостоянства, так как период цикличности является случайной
величиной и матожидание, дисперсия и ковариация данного процесса будут константными.

Давайте посмотрим на пример ниже. В изначальном ряде на обоих графиках мы наблюдаем и
тренд и сезонность и гетероскедастичность. Данный ряд явно не является стационарным.

In [ ]:
ts = ds['international-airline-passengers.csv']

plot_ts(ts)
plot_acf(ts);

Теперь рассмотрим график белого шума.

In [ ]:
ts = pd.Series(
    data=[randint(0, 100) for _ in range(100)],
    index=pd.date_range(start='2022-08-05', freq='1H', periods=100)
)

plot_ts(ts)
plot_acf(ts);

Здесь мы не видим никакого тренда, сезонности или гетероскедастичности. А график
автокорреляции показывает резкий спад корреляции в незначимый диапазон сразу после 0-го
лага. Этот ряд явно стационарен.

*Статистические тесты*

Вторым способом является проверка ряда на стационарность при помощи какого-нибудь
статистического теста. Два самых популярных теста для проверки ряда на стационарность -
тест [KPSS](#ref_3) и [Dicky-Fuller](#ref_4). Мы будем использовать второй. В рамках данного обзора мы не
будем вдаваться в теорию, стоящую за данными тестами, и будем использовать их как "black
box".

Нулевой гипотезой теста 

Тем не менее, автоматического способа, который со 100% гарантией давал бы вам вердикт о
стационарности не существует, к тому же условие стационарности, как и любое другое
статистическое понятие, может определяться лишь с некоей вероятностью. Поэтому этот
требует некоего опыта.

## <a id='time_series_transformations'>Преобразования временных рядов</a>

### *Временные преобразования временного ряда*

Представьте, что вам дали задачу спрогнозировать среднегодовую минимальную температуру в 
определенном районе, при этом у вас имеются данные минимальной температуры за каждый день
в течении нескольких лет. Для того чтобы спрогнозировать температуру на следующий год, вам
необходимо сделать прогноз на 365 дней вперед и потом усреднить значение. Тем не менее,
такая задача может быть весьма сложной. Однако, на самом деле, вам не нужно знать
температуру на каждый день - вместо этого вы можете взять среднее значение за каждый год и
спрогнозировать уже этот, новый, годичный ряд. Таким образом перейти от дневной
гранулярности к годичной. Такие преобразования называется временными.

>Попробуйте расчитать среднее значение температуры на следующий год для ряда
>`daily-min-temperatures.csv`, используя временное преобразование. Для этого рассчитайте
>среднее за каждый год и перейдите к годичному ряду, после чего посмотрите на ряд и оцените
>среднее значение температуры на следующий год.

In [ ]:
ts = ds['daily-min-temperatures.csv']
# your code here
years = {dt.year for dt in ts.index}

year_min_temperatures = {
    str(year): ts[f'{year}'].mean()
    for year in years
}

ts_year = pd.Series(year_min_temperatures)

ts_year.index = pd.to_datetime(ts_year.index)

ts_year.sort_index(inplace=True)

plot_ts(ts_year)

# видим, что оптимальным прогнозом температуры на 91 год будет значение где-то от 11.3 до 12

### *Логарифмирование временного ряда*

Самым простым способом убрать непостоянство дисперсии во временном ряде является операция
логарифмирования.

$$y_t^{log} = log{y_t}$$

>Попробуйте применить операцию логарифмирования к ряду `alcohol_sales.csv` и посмотрите как
>изменится ряд. Получилось ли убрать непостоянство дисперсии?

In [ ]:
ts = ds['alcohol_sales.csv']
# your code here

import numpy as np

plot_ts(ts)
plot_ts(np.log(ts))
# видим, что непостоянство дисперсии действительно ушло

Развитием идеи логарифмирования ряда является преобразование Бокса-Кокса, которое
предполагает трансформация ряда при помощи произвольной степенной функции. Данное
преобразование мы оставим за рамками данного обзора, можете ознакомиться с ним
самостоятельно [здесь](#ref_2).

### *Дифференцирование временного ряда (или взятие разностей)*

Убрать тренд или сезонность проще всего при помощи дифференцирования временного ряда, т.е. перейдя от асболютных величин к их разностям, кроме того, работая с временными рядами, очень часто хочется посмотреть не на сам ряд, а на его изменение. Чтобы можно было сказать, насколько ряд меняется с каждой новой точкой, или,
скажем, насколько у дневного ряда различаются значения вчера и сегодня.

Пусть имеется ряд $y_t$, тогда, дифференцированным с лагом $k$ будет называться ряд вида
$y_t^{'} = y_t - y_{t-k}$

Для операции дифференцирования также часто вводят понятие порядка $d$. Дифференцированным
рядом порядка $d$ с лагом $k$ называется ряд, который $d$ раз продифференцирован с лагом
$k$.

В питоне операцию дифференцирования легко осуществить при помощи метода `diff` класса
TimeSeries.

In [ ]:
ts = ds['international-airline-passengers.csv']

# взятие первой разности с лагом 1
ts.diff()

# взятие первой разности с лагом 12
ts.diff(12)

# взятие второй разности с лагом 1
ts.diff().diff()

# можно также комбинировать порядок и лаг
ts.diff().diff(12)

# посмотрим, как все выглядит на графике
plot_ts(ts, ts.diff(), ts.diff(12))

>Попробуйте избавить ряд `daily-min-temperatures` от сезонности при помощи операции дифференцирования

In [ ]:
ts = ds['daily-min-temperatures.csv']
# your code here

ts_diff = ts.diff(365)

plot_ts(ts, ts_diff)

# <a id='forecasting'>Прогнозирование</a>

# <a id='anomaly_detection'>Поиск аномалий</a>

# <a id='time_series_classification'>Классификация временных рядов</a>

References.

<a id='ref_1'>[1]: [ISO durations](https://en.wikipedia.org/wiki/ISO_8601#Durations)</a>  
<a id='ref_2'>[2]: [Box-Cox transformation](http://www.machinelearning.ru/wiki/index.php?title=Метод_Бокса-Кокса)</a>